# Introducción práctica a redes neuronales densas en PyTorch con MNIST

### *Universidad Nacional Autónoma de México*
## Autor: **Sebastián González Juárez**

### Idea general

**Objetivo**: entrenar una MLP (perceptrón multicapa) sencilla sobre MNIST para clasificar dígitos (0–9).

**Flujo**:

- Carga y normalización: se descargan las imágenes con torchvision.datasets.MNIST, se convierten a tensores en [0,1] y se aplanan de 1×28×28 a 784.

- Batches: se crean DataLoaders con batch_size=128 (barajando el entrenamiento).

- Modelo: Linear(784→512) → ReLU → Linear(512→10).

- La última capa no usa Softmax; entrega logits.

- Pérdida y optimización: nn.CrossEntropyLoss (que integra LogSoftmax) + Adam (lr=1e-3).

- Entrenamiento (5 épocas): bucle típico forward → loss → backward → step, registrando loss y accuracy por época.

- Predicción rápida: se obtienen probabilidades con F.softmax para inspeccionar un ejemplo.

- Evaluación: se calcula la exactitud en test recorriendo el test_loader.

Imports:

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset

## 1) Datos.

Cargar MNIST y preparar como (N, 784) float32/255


In [2]:
tfms = transforms.ToTensor()  # [0,255] -> float32 [0,1]
train_ds = datasets.MNIST(root='./data', train=True,  download=True, transform=tfms)
test_ds  = datasets.MNIST(root='./data', train=False, download=True, transform=tfms)

100%|██████████| 9.91M/9.91M [00:00<00:00, 19.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 500kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.46MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 12.1MB/s]


Convertimos a tensores (N,784) para imitar tu reshape de Keras


In [4]:
train_images = torch.stack([train_ds[i][0] for i in range(len(train_ds))])  # (60000,1,28,28)
train_labels = torch.tensor([train_ds[i][1] for i in range(len(train_ds))]) # (60000,)
test_images  = torch.stack([test_ds[i][0]  for i in range(len(test_ds))])   # (10000,1,28,28)
test_labels  = torch.tensor([test_ds[i][1] for i in range(len(test_ds))])   # (10000,)

In [5]:
train_images = train_images.view(60000, 28*28)  # reshape a (60000, 784)
test_images  = test_images.view(10000,  28*28)  # reshape a (10000, 784)
# Nota: transforms.ToTensor() ya normalizó a [0,1], así que no dividimos de nuevo por 255.

DataLoaders con batch_size=128.

In [6]:
train_loader = DataLoader(TensorDataset(train_images, train_labels), batch_size=128, shuffle=True)
test_loader  = DataLoader(TensorDataset(test_images,  test_labels),  batch_size=128, shuffle=False)

## 2) Modelo:

Dense(512,relu)->Dense(10,softmax)

En PyTorch, si usamos CrossEntropyLoss NO ponemos softmax en la capa final (la loss lo aplica internamente).

In [7]:
model = nn.Sequential(
    nn.Linear(28*28, 512),
    nn.ReLU(),
    nn.Linear(512, 10)
)

##3) Compilación:

optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]


In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

##4) Entrenamiento:

epochs=5, batch_size=128 (equivale a model.fit(...))


In [9]:
for epoch in range(5):
    model.train()
    correct, total, running_loss = 0, 0, 0.0
    for xb, yb in train_loader:
        logits = model(xb)                 # forward
        loss = criterion(logits, yb)       # CE con labels enteros (sparse)
        optimizer.zero_grad()
        loss.backward()                    # backward
        optimizer.step()                   # update

        running_loss += loss.item() * xb.size(0)
        preds = logits.argmax(dim=1)
        correct += (preds == yb).sum().item()
        total   += yb.size(0)

    print(f"Epoch {epoch+1} - loss: {running_loss/total:.4f} - acc: {correct/total:.4f}")

Epoch 1 - loss: 0.3141 - acc: 0.9131
Epoch 2 - loss: 0.1251 - acc: 0.9642
Epoch 3 - loss: 0.0815 - acc: 0.9764
Epoch 4 - loss: 0.0584 - acc: 0.9829
Epoch 5 - loss: 0.0432 - acc: 0.9871


## 5) Predicción rápida de los primeros 10 test (para emular predict + argmax)


In [10]:
model.eval()
with torch.no_grad():
    test_digits = test_images[:10]                    # (10, 784)
    logits = model(test_digits)                       # (10, 10)
    probs  = F.softmax(logits, dim=1)                 # para ver probabilidades (como tu softmax)
    first_probs = probs[0]
    print("predictions[0] =", first_probs.numpy())
    print("predictions[0].argmax() =", first_probs.argmax().item())
    print("predictions[0][7] =", first_probs[7].item())
    print("test_labels[0] =", test_labels[0].item())

predictions[0] = [1.7907246e-07 5.2774539e-08 7.7493067e-05 7.9781213e-04 1.3021066e-09
 8.2873055e-07 1.6382718e-12 9.9909580e-01 1.2581635e-05 1.5155536e-05]
predictions[0].argmax() = 7
predictions[0][7] = 0.9990957975387573
test_labels[0] = 7


##6) Evaluación (equivale a model.evaluate)

In [11]:
model.eval()
correct, total, running_loss = 0, 0, 0.0
with torch.no_grad():
    for xb, yb in test_loader:
        logits = model(xb)
        loss = criterion(logits, yb)
        running_loss += loss.item() * xb.size(0)
        preds = logits.argmax(dim=1)
        correct += (preds == yb).sum().item()
        total   += yb.size(0)

In [12]:
test_acc = correct / total
print(f"test_acc: {test_acc:.4f}")

test_acc: 0.9804


## Conclusiones

Con muy poco código se logra una línea base fuerte en MNIST usando PyTorch idiomático.

- Separar logits (para la pérdida) y probabilidades (solo para inspección) evita inestabilidades y es la práctica recomendada.

- DataLoader + Adam simplifican el entrenamiento y suelen converger rápido con una MLP pequeña.

- Esta versión es más robusta y eficiente que una implementación “desde cero”, y sirve como punto de partida para:

  - Añadir regularización (Dropout, Weight Decay),

  - Probar arquitecturas (más capas, otras activaciones),

  - Cambiar a CNNs para exprimir mejor la estructura espacial.